# Задание
Возьмите данные отзывов о фильмах сайта Кинопоиск ру из соревнования https://www.kaggle.com/mikhailklemin/kinopoisks-movies-reviews 

Обучите реккурентную нейронную сеть с различными вариантами embeddig:
* собственный embedding
* word2vec
* GLOVE

после чего дообучите нейронную сеть Bert на
* 10%
* 20%
* 50% 
обучающих примеров

Попробуйте добиться точности выше, чем у рекуррентной нейронной сети.

## Import

In [1]:
from pathlib import Path
import numpy as np
import pandas as pd

# BERT imports
import torch
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup

# specify GPU device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available():
    n_gpu = torch.cuda.device_count()
    torch.cuda.get_device_name(0)

In [2]:
dataset_path = Path('../datasets/kinopoisk/')

##  Utils

In [3]:
def standardize_text(df, content_field):
    df[content_field] = df[content_field].str.replace(r"http\S+", "")
    df[content_field] = df[content_field].str.replace(r"@\S+", "")
    df[content_field] = df[content_field].str.replace(
        r"[^А-Яа-яA-Za-z0-9Ёё(),!?@\'\`\"\_\n]", " ")
    df[content_field] = df[content_field].str.replace(r"[Ёё]", "е")
    df[content_field] = df[content_field].str.replace(r"[\t\n]", "")
    df[content_field] = df[content_field].str.replace(r"[^А-Яа-яa-zA-Z]", " ")
    df[content_field] = df[content_field].str.lower()
    return df

In [4]:
class_names = ["Negative", "Neutral", "Positive"]
class_names_converter = {
    'neg': 'Negative',
    'pos': 'Positive',
    'neu': 'Neutral',
}

def name_to_id(name):
    return class_names.index(name)

### Класс для токенизации

In [5]:
class BertDataset(Dataset):
    def __init__(self, texts, targets, tokenizer, max_len=512):
        self.texts = texts
        self.targets = targets
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.class_weights = {name: 1-(count/len(self)) for name, count in self.targets.value_counts().items()}

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        if isinstance(idx, int):
            idx = idx if idx < len(self) else len(self)-1
        text = str(self.texts[idx])
        target = self.targets[idx]

        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt',
            truncation=True
        )

        return {
          'text': text,
          'input_ids': encoding['input_ids'].flatten(),
          'attention_mask': encoding['attention_mask'].flatten(),
          'targets': torch.tensor(target, dtype=torch.long)
        }

    def get_weights(self):
        return [self.class_weights[sample] for sample in self.targets]

### Класс для обучения

In [6]:
class BertClassifier:
    def __init__(
        self,model, 
        tokenizer, optimizer, 
        loss, max_len=512, 
        model_save_path='rubert_on_kinopoisk.pt', log=True
    ):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        
        self.model = model
        self.model.to(self.device)
        
        self.tokenizer = tokenizer
        self.optimizer = optimizer

        self.loss = loss
        self.loss.to(self.device)

        self.model_save_path=model_save_path
        self.max_len = max_len
        self.log = log
    
    def fit(self, train, eval, epochs=2, save_model=True):
        self.model = self.model.train()
        losses = []
        correct_predictions = 0
        best_accuracy = 0

        train_dl = DataLoader(train, batch_size=8, sampler=torch.utils.data.RandomSampler(train))

        scheduler = get_linear_schedule_with_warmup(
                self.optimizer,
                num_warmup_steps=0,
                num_training_steps=len(train_dl) * epochs
            )

        for epoch in range(epochs):

            correct_predictions = 0
            for data in train_dl:

                input_ids = data["input_ids"].to(self.device)
                attention_mask = data["attention_mask"].to(self.device)
                targets = data["targets"].to(self.device)

                outputs = self.model(
                    input_ids=input_ids,
                    attention_mask=attention_mask
                    )

                preds = torch.argmax(outputs.logits, dim=1)
                loss = self.loss(outputs.logits, targets)

                correct_predictions += torch.sum(preds == targets)

                losses.append(loss.item())
                
                loss.backward()
                torch.nn.utils.clip_grad_norm_(self.model.parameters(), max_norm=1.0)
                self.optimizer.step()
                scheduler.step()
                self.optimizer.zero_grad()

            train_acc = correct_predictions.double() / len(train)
            train_loss = np.mean(losses)

            eval_acc, eval_loss = self.eval(eval)
            
            if self.log:
                print(f' Epoch {epoch+1}/{epochs}')
                print(f'Train loss: {train_loss:.4f} accuracy: {train_acc:.4f}')
                print(f'Val loss {eval_loss:.4f} accuracy {eval_acc:.4f}')

            if save_model and eval_acc > best_accuracy:
                torch.save(self.model, self.model_save_path)
                best_accuracy = eval_acc

        return self 

    def eval(self, eval):
        self.model = self.model.eval()
        losses = []
        correct_predictions = 0

        eval_dl = DataLoader(eval, batch_size=32, sampler=torch.utils.data.SequentialSampler(eval))
        with torch.no_grad():
            for data in eval_dl:
                input_ids = data["input_ids"].to(self.device)
                attention_mask = data["attention_mask"].to(self.device)
                targets = data["targets"].to(self.device)

                outputs = self.model(
                    input_ids=input_ids,
                    attention_mask=attention_mask
                    )

                preds = torch.argmax(outputs.logits, dim=1)
                loss = self.loss(outputs.logits, targets)
                correct_predictions += torch.sum(preds == targets)
                losses.append(loss.item())
        
        val_acc = correct_predictions.double() / len(eval)
        val_loss = np.mean(losses)
        return val_acc, val_loss
    
    def predict(self, text):
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            truncation=True,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt',
        )
        
        out = {
              'text': text,
              'input_ids': encoding['input_ids'].flatten(),
              'attention_mask': encoding['attention_mask'].flatten()
          }
        
        input_ids = out["input_ids"].to(self.device)
        attention_mask = out["attention_mask"].to(self.device)
        
        outputs = self.model(
            input_ids=input_ids.unsqueeze(0),
            attention_mask=attention_mask.unsqueeze(0)
        )
        
        prediction = torch.argmax(outputs.logits, dim=1).cpu().numpy()[0]

        return prediction

## Pre-Convert Datasets

In [16]:
for perc in [0.1, 0.2, 0.5, 1]:
    df = pd.DataFrame(columns=['review', 'sentiment'])
    
    for class_path in dataset_path.iterdir():
        if class_path.is_dir():
            dirs = np.array(list(class_path.iterdir()))
            np.random.shuffle(dirs)
            rews_fhs = np.random.choice(dirs, round(len(dirs)*perc))
            print(f'len = {rews_fhs.shape}')
            print(class_names_converter[class_path.name])
            for rew_fh in rews_fhs:
                with open(Path(rew_fh), encoding='utf-8') as f:
                    review = f.read()
                    current_df = pd.DataFrame(
                        {'review': [review], 'sentiment': class_names_converter[class_path.name]})
                    df = pd.concat([df, current_df], ignore_index=True)

    df = df.sample(frac=1).reset_index(drop=True)

    df = standardize_text(df, "review")
    df['sentiment'] = df['sentiment'].map(name_to_id)

    train_dataset, eval_dataset = train_test_split(df, test_size = 0.2)

    train_dataset.to_csv(f'Kinopoisk_train_{perc:.0%}.csv')
    eval_dataset.to_csv(f'Kinopoisk_eval_{perc:.0%}.csv')

len = (1983,)
Negative
len = (2470,)
Neutral
len = (8714,)
Positive


C:\Users\leysh\AppData\Local\Temp\ipykernel_13496\323947140.py:11: FutureWarning: The default value of regex will change from True to False in a future version.
  df[content_field] = df[content_field].str.replace(r"http\S+", "")
C:\Users\leysh\AppData\Local\Temp\ipykernel_13496\323947140.py:12: FutureWarning: The default value of regex will change from True to False in a future version.
  df[content_field] = df[content_field].str.replace(r"@\S+", "")
C:\Users\leysh\AppData\Local\Temp\ipykernel_13496\323947140.py:13: FutureWarning: The default value of regex will change from True to False in a future version.
  df[content_field] = df[content_field].str.replace(
C:\Users\leysh\AppData\Local\Temp\ipykernel_13496\323947140.py:15: FutureWarning: The default value of regex will change from True to False in a future version.
  df[content_field] = df[content_field].str.replace(r"[Ёё]", "е")
C:\Users\leysh\AppData\Local\Temp\ipykernel_13496\323947140.py:16: FutureWarning: The default value of r

len = (3965,)
Negative
len = (4941,)
Neutral
len = (17428,)
Positive


C:\Users\leysh\AppData\Local\Temp\ipykernel_13496\323947140.py:11: FutureWarning: The default value of regex will change from True to False in a future version.
  df[content_field] = df[content_field].str.replace(r"http\S+", "")
C:\Users\leysh\AppData\Local\Temp\ipykernel_13496\323947140.py:12: FutureWarning: The default value of regex will change from True to False in a future version.
  df[content_field] = df[content_field].str.replace(r"@\S+", "")
C:\Users\leysh\AppData\Local\Temp\ipykernel_13496\323947140.py:13: FutureWarning: The default value of regex will change from True to False in a future version.
  df[content_field] = df[content_field].str.replace(
C:\Users\leysh\AppData\Local\Temp\ipykernel_13496\323947140.py:15: FutureWarning: The default value of regex will change from True to False in a future version.
  df[content_field] = df[content_field].str.replace(r"[Ёё]", "е")
C:\Users\leysh\AppData\Local\Temp\ipykernel_13496\323947140.py:16: FutureWarning: The default value of r

len = (9914,)
Negative
len = (12352,)
Neutral
len = (43569,)
Positive


C:\Users\leysh\AppData\Local\Temp\ipykernel_13496\323947140.py:11: FutureWarning: The default value of regex will change from True to False in a future version.
  df[content_field] = df[content_field].str.replace(r"http\S+", "")
C:\Users\leysh\AppData\Local\Temp\ipykernel_13496\323947140.py:12: FutureWarning: The default value of regex will change from True to False in a future version.
  df[content_field] = df[content_field].str.replace(r"@\S+", "")
C:\Users\leysh\AppData\Local\Temp\ipykernel_13496\323947140.py:13: FutureWarning: The default value of regex will change from True to False in a future version.
  df[content_field] = df[content_field].str.replace(
C:\Users\leysh\AppData\Local\Temp\ipykernel_13496\323947140.py:15: FutureWarning: The default value of regex will change from True to False in a future version.
  df[content_field] = df[content_field].str.replace(r"[Ёё]", "е")
C:\Users\leysh\AppData\Local\Temp\ipykernel_13496\323947140.py:16: FutureWarning: The default value of r

## Load Prepeared Datasets

In [7]:
train_datasets = []
eval_datasets = []

for perc in [0.1, 0.2, 0.5, 1]:
    train_dataset = pd.read_csv(f'Kinopoisk_train_{perc:.0%}.csv')
    eval_dataset = pd.read_csv(f'Kinopoisk_eval_{perc:.0%}.csv')
    train_datasets.append(train_dataset)
    eval_datasets.append(eval_dataset)

### Tokenizer

In [8]:
rubert_path = 'cointegrated/rubert-tiny2'
tokenizer = BertTokenizer.from_pretrained(rubert_path)

In [9]:
train_datasets = [BertDataset(dataset['review'], dataset['sentiment'], tokenizer, max_len=512) for dataset in train_datasets]
eval_datasets = [BertDataset(dataset['review'], dataset['sentiment'], tokenizer, max_len=512) for dataset in eval_datasets]

## Load Model

In [10]:
model = BertForSequenceClassification.from_pretrained(rubert_path)

out_features = model.bert.encoder.layer[1].output.dense.out_features
model.classifier = torch.nn.Linear(out_features, len(class_names))

Some weights of the model checkpoint at cointegrated/rubert-tiny2 were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not 

In [11]:
optimizer = AdamW(model.parameters(), lr=2e-5, correct_bias=False)

loss_fn = torch.nn.CrossEntropyLoss().to(device)

c:\Users\leysh\anaconda3\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


## Train

In [12]:
num_epochs = [1, 2, 2, 4]
model_names = ['ds_10%', 'ds_20%', 'ds_50%', 'ds_100%']

for num_epochs, model_name, train_ds, eval_ds in zip(num_epochs, model_names, train_datasets, eval_datasets):
    model = BertForSequenceClassification.from_pretrained(rubert_path)

    out_features = model.bert.encoder.layer[1].output.dense.out_features
    model.classifier = torch.nn.Linear(out_features, len(class_names))

    optimizer = AdamW(model.parameters(), lr=2e-5, correct_bias=False)
    loss_fn = torch.nn.CrossEntropyLoss().to(device)

    BertClassifier(model, tokenizer, optimizer, loss_fn, model_save_path=f'rubert_on_kinopoisk_{model_name}.pt').fit(train_ds, eval_ds, epochs=num_epochs)

Some weights of the model checkpoint at cointegrated/rubert-tiny2 were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not 

ValueError: Unable to create tensor, you should probably activate truncation and/or padding with 'padding=True' 'truncation=True' to have batched tensors with the same length. Perhaps your features (`input_ids` in this case) have excessive nesting (inputs type `list` where type `int` is expected).

## Eval

In [28]:
for model_name in model_names:
    model = torch.load(f'rubert_on_kinopoisk_{model_name}.pt')
    
    acc, loss = BertClassifier(model, tokenizer, optimizer, loss_fn).eval(eval_datasets[-1])
    print(f'({model_name}) model\'s accuracy is === {acc:.2%} === with average loss = {loss:.5f}' )

(ds_10%) model's accuracy is === 72.70% === with average loss = 0.65089


## Check 

In [32]:
for suffix in model_names:
# Отрицательный отзыв на фильм Морбиус (2022) 
    model = torch.load(f'rubert_on_kinopoisk_{suffix}.pt')
    model_runner = BertClassifier(model, tokenizer, optimizer, loss_fn)
    print(f'======================= ruBert with {suffix} dataset volume =========================')
    header = "Morbius 2022"
    review = "Ну, я даже не знаю, какой супергеройский фильм последних лет может хотя бы на толику быть на столько ужасным. Абсолютно не спасает этот плевок в сторону фан-сообщества наличие нестареющего (видимо действительно вампирского происхождения) Джареда Лето. Хотя, чего уж скрывать, он всё равно красавчик, и эта роль ему на все сто процентов подходит, НО только при наличии адекватного сценария и не отвлекающегося на перекуры и другие интересные дела режиссера"
    correct = "Negative"
    model_out = model_runner.predict(review)
    print(f'{"V" if class_names[model_out]==correct else "X"} Результат модели: This review about "{header}" is {class_names[model_out]} (in fact it is {correct})')

# Положительный Avengers: Endgame, 2019
    header = "Avengers: Endgame, 2019"
    review = "Есть фильмы, которые хороши не потому, что в них все идеально. Картина может иметь сотню недостатков, куча дыр и несостыковок, завышенные ожидания со стороны зрителей, но все равно цепляет и оставляет по себе очень приятное послевкусие. Именно к таким фильмам лично я для себя причисляю Мстителей. Финал'. Просматривая его третий раз в кино, я поняла, что еще раз и смогу разложить и сюжет и мотивацию героев по атомам, тем не менее, ни ненависти ни какого-то огорчения я не испытываю. Если бы меня спросили, что бы я поменяла, то определенно получилось бы эссе на несколько страничек мелкого почерка. С другой стороны, мы получили весьма зрелищное и душевное окончание многолетней саги, увидели любимых персонажей, вдоволь посмеялись и даже местами поплакали. А раз фильм был способен вызвать такую гамму эмоций, значит создатели сделали почти все правильно."
    correct = "Positive"
    model_out = model_runner.predict(review)
    print(f'{"V" if class_names[model_out]==correct else "X"} Результат модели: This review about "{header}" is {class_names[model_out]} (in fact it is {correct})')

# Отрицательный Justice League, 2017
    header = "Justice League, 2017"
    review = "И тут все дело в подаче. Понятно что в комиксах вселенная DC давно существует и там Бэтмен и Лига Справедливости спина к спине оберегают Землю. Но в кино мире - Бэтмен всегда был одиночкой, и соперники у него были под стать - без исключительных суперспособностей. И теперь DC пытается впихнуть героя, который в массовом сознании прослыл 'одиноким рейнджером' к героям у которых настоящие суперсилы и суперспособности. Бэтмен на их фоне выглядит ну совсем никак. Но даже не смотря на диссонанс с Бэтменом у DC большие проблемы: 'Лига справедливости', по сравнению с 'Мстителями' выглядит просто дешевой поделкой, вроде все как у них но нет того ощущения постоянного драйва, герои раскрыты однобоко, а вечный пафос раздражает - так как ощущается чем то инородным и неуместным к данной картине. Сюжет в Лиге очень плоский, спецэффекты на уровне, но если их не сдабривать нужным эмоциональным фоном, они начинают смотреться как нарезка трюков. Конечно были в фильмы проблески чего то хорошего, но на общем фоне картины они не осели в памяти, зато осело разочарование от завышенных ожиданий"
    correct = "Negative"
    model_out = model_runner.predict(review)
    print(f'{"V" if class_names[model_out]==correct else "X"} Результат модели: This review about "{header}" is {class_names[model_out]} (in fact it is {correct})')

======================= ruBert with ds_10% dataset volume =========================
X Результат модели: This review about "Morbius 2022" is Neutral (in fact it is Negative)
V Результат модели: This review about "Avengers: Endgame, 2019" is Positive (in fact it is Positive)
X Результат модели: This review about "Justice League, 2017" is Neutral (in fact it is Negative)
======================= ruBert with ds_20% dataset volume =========================
X Результат модели: This review about "Morbius 2022" is Neutral (in fact it is Negative)
V Результат модели: This review about "Avengers: Endgame, 2019" is Positive (in fact it is Positive)
V Результат модели: This review about "Justice League, 2017" is Negative (in fact it is Negative)
======================= ruBert with ds_50% dataset volume =========================
X Результат модели: This review about "Morbius 2022" is Neutral (in fact it is Negative)
V Результат модели: This review about "Avengers: Endgame, 2019" is Positive (in fact

Результат модели LSTM, созданной ранее в [ДЗ](https://colab.research.google.com/drive/1uSbb1xCYA3M2144oq0rBUdJN_NcTskED?usp=sharing)  показал точность в 73% на тестовой выборке.  
В итоге получили точность модели BERT с предобучением и токенизатором от ruBERT   в 86% при обучении на 100% датасете. Обучение на 10% размере датасета показывает такую же точность модели как и LTSM.